***
# Model Deployment : Detecting and Analyzing Machine Learning Model Drift Using Open-Source Monitoring Tools

***
### [**John Pauline Pineda**](https://github.com/JohnPaulinePineda) <br> <br> *September 30, 2025*
***

* [**1. Table of Contents**](#TOC)
    * [1.1 Data Background](#1.1)
    * [1.2 Data Description](#1.2)
    * [1.3 Data Quality Assessment](#1.3)
    * [1.4 Data Preprocessing](#1.4)
        * [1.4.1 Data Splitting](#1.4.1)
        * [1.4.2 Outlier and Distributional Shape Analysis](#1.4.4)
        * [1.4.3 Collinearity](#1.4.5)
    * [1.5 Data Exploration](#1.5)
        * [1.5.1 Exploratory Data Analysis](#1.5.1)
        * [1.5.2 Hypothesis Testing](#1.5.2)
    * [1.6 Premodelling Data Preparation](#1.6)
        * [1.6.1 Preprocessed Data Description](#1.6.1)
        * [1.6.2 Preprocessing Pipeline Development](#1.6.2)
    * [1.7 Model Development and Validation](#1.7)
        * [1.7.1 Random Forest](#1.7.1)
        * [1.7.2 AdaBoost](#1.7.2)
        * [1.7.3 Gradient Boosting](#1.7.3)
        * [1.7.4 XGBoost](#1.7.4)
        * [1.7.5 Light GBM](#1.7.5)
        * [1.7.6 CatBoost](#1.7.6)
    * [1.8 Model Monitoring using the NannyML Framework](#1.8)
        * [1.8.1 Baseline Control](#1.8.1)
        * [1.8.2 Simulated Covariate Drift](#1.8.2)
        * [1.8.3 Simulated Prior Shift](#1.8.3)
        * [1.8.4 Simulated Concept Drift](#1.8.4)
        * [1.8.5 Simulated Missingness Spike](#1.8.5)
        * [1.8.6 Simulated Seasonal Pattern](#1.8.6)
* [**2. Summary**](#Summary)   
* [**3. References**](#References)

***

# 1. Table of Contents <a class="anchor" id="TOC"></a>

## 1.1. Data Background <a class="anchor" id="1.1"></a>

## 1.2. Data Description <a class="anchor" id="1.2"></a>

## 1.3. Data Quality Assessment <a class="anchor" id="1.3"></a>

## 1.4. Data Preprocessing <a class="anchor" id="1.4"></a>

### 1.4.1 Data Splitting<a class="anchor" id="1.4.1"></a>

### 1.4.2 Outlier and Distributional Shape Analysis<a class="anchor" id="1.4.2"></a>

### 1.4.3 Collinearity<a class="anchor" id="1.4.3"></a>

## 1.5. Data Exploration <a class="anchor" id="1.5"></a>

### 1.5.1 Exploratory Data Analysis<a class="anchor" id="1.5.1"></a>

### 1.5.2 Hypothesis Testing<a class="anchor" id="1.5.2"></a>

## 1.6. Premodelling Data Preparation <a class="anchor" id="1.6"></a>

### 1.6.1 Preprocessed Data Description<a class="anchor" id="1.6.1"></a>

### 1.6.2 Preprocessing Pipeline Development<a class="anchor" id="1.6.2"></a>

## 1.7. Model Development and Validation <a class="anchor" id="1.7"></a>

### 1.7.1 Random Forest<a class="anchor" id="1.7.1"></a>

### 1.7.2 AdaBoost<a class="anchor" id="1.7.2"></a>

### 1.7.3 Gradient Boosting<a class="anchor" id="1.7.3"></a>

### 1.7.4 XGBoost<a class="anchor" id="1.7.4"></a>

### 1.7.5 Light GBM<a class="anchor" id="1.7.5"></a>

### 1.7.6 CatBoost<a class="anchor" id="1.7.6"></a>

## 1.8. Model Monitoring using the NannyML Framework <a class="anchor" id="1.8"></a>

### 1.8.1 Baseline Control<a class="anchor" id="1.8.1"></a>

### 1.8.2 Simulated Covariate Drift<a class="anchor" id="1.8.2"></a>

### 1.8.3 Simulated Prior Shift<a class="anchor" id="1.8.3"></a>

### 1.8.4 Simulated Concept Drift<a class="anchor" id="1.8.4"></a>

### 1.8.5 Simulated Missingness Spike<a class="anchor" id="1.8.5"></a>

### 1.8.6 Simulated Seasonal Pattern<a class="anchor" id="1.8.6"></a>

## 1.9. Consolidated Findings <a class="anchor" id="1.9"></a>

# 2. Summary <a class="anchor" id="Summary"></a>

# 3. References <a class="anchor" id="References"></a>